In [6]:
import pandas as pd
import numpy as np
import surprise
import apyori
from surprise import accuracy
from surprise import SVD
from surprise.model_selection.search import GridSearchCV

In [21]:
data = surprise.Dataset.load_builtin("ml-100k")
trainingSet = data.build_full_trainset()
testSet = trainingSet.build_testset()

In [22]:
random_state = [x for x in range(1,5)]

In [23]:
svd = SVD()

In [24]:
param_grid = {'random_state': random_state}

In [28]:
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], n_jobs=-1, cv=)

In [29]:
gs.fit(data)

In [30]:
gs.best_params

{'rmse': {'random_state': 4}}

In [31]:
gs.best_score

{'rmse': 0.9369045894110132}

In [32]:
gs.cv_results

{'split0_test_rmse': array([0.93572402, 0.93497494, 0.9329611 , 0.9346868 ]),
 'split1_test_rmse': array([0.93426217, 0.93955535, 0.93486875, 0.9354199 ]),
 'split2_test_rmse': array([0.94214847, 0.94375311, 0.94397207, 0.94333726]),
 'split3_test_rmse': array([0.93617199, 0.93424363, 0.93422895, 0.93289646]),
 'split4_test_rmse': array([0.93878351, 0.93860976, 0.94041796, 0.93818252]),
 'mean_test_rmse': array([0.93741803, 0.93822736, 0.93728977, 0.93690459]),
 'std_test_rmse': array([0.00277933, 0.00343197, 0.0042049 , 0.00363869]),
 'rank_test_rmse': array([3, 4, 2, 1], dtype=int64),
 'mean_fit_time': array([7.78228788, 8.11182852, 7.97001801, 8.37054238]),
 'std_fit_time': array([0.01640026, 0.16674577, 0.30925376, 0.49412362]),
 'mean_test_time': array([0.28922491, 0.27487111, 0.28205771, 0.2920094 ]),
 'std_test_time': array([0.07730595, 0.07866192, 0.06249422, 0.07415257]),
 'params': [{'random_state': 1},
  {'random_state': 2},
  {'random_state': 3},
  {'random_state': 4}],
 'p

In [33]:
i=1
svd=SVD(random_state=i)
svd.fit(trainingSet)
predictions_svd=svd.test(testSet)
predictions_svd[42]

Prediction(uid='186', iid='148', r_ui=4.0, est=3.563479670883666, details={'was_impossible': False})

In [34]:
accuracy.rmse(predictions_svd)

RMSE: 0.6757


0.6756643049127705

In [5]:
trainingSet.n_users

943

In [6]:
trainingSet.n_items

1682

In [7]:
trainingSet.global_mean

3.52986

In [3]:
from surprise import KNNBasic

sim_options = {
    'name': 'cosine',
    'user_based': False
}
 
knn = KNNBasic(sim_options=sim_options)

knn.fit(trainingSet)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [4]:
predictions = knn.test(testSet)

In [5]:
predictions[8]

Prediction(uid='196', iid='663', r_ui=5.0, est=3.6206349533418014, details={'actual_k': 39, 'was_impossible': False})

In [4]:
4 - 3.575883908398154

0.4241160916018458

In [7]:
from surprise import KNNBasic

sim_options = {
    'name': 'cosine',
    'user_based': True
}
 
knn = KNNBasic(sim_options=sim_options)

knn.fit(trainingSet)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [8]:
pred_knn = knn.test(testSet)

In [26]:
accuracy.rmse(pred_knn)

RMSE: 0.9012


0.9012175824620821

In [89]:
df = pd.DataFrame(data.raw_ratings)
df.columns = ['user_id', 'movie_id', 'rating', 3]
df.head()

,user_id,movie_id,rating,3
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [102]:
good = df[df.rating >= 4].groupby('user_id')['movie_id'].apply(lambda x: ' '.join([str(A) for A in x]))

In [117]:
good['100']

'344 355 750 302 691 316 752 313 879 300 328 1235 690 347 269 258 900 294 272 751 898 315'

In [122]:
association_rules = apyori.apriori(good.apply(lambda r: r.split(' ')), 
                                   min_support=0.2, 
                                   min_confidence=0.3, min_lift=2, 
                                   min_length=2)

In [123]:
asr_df = pd.DataFrame(columns = ['from', 'to', 'confidence', 'support', 'lift'])
for item in association_rules:
    pair = item[0] 
    items = [x for x in pair]
    asr_df.loc[len(asr_df), :] =  ' '.join(list(item[2][0][0])), \
                                  ' '.join(list(item[2][0][1])),\
                                  item[2][0][2], item[1], item[2][0][3]

In [125]:
asr_df

,from,to,confidence,support,lift
0,172,174,0.819113,0.254777,2.21725
1,173,174,0.762097,0.200637,2.06292
2,174,195,0.548851,0.20276,2.38257
3,174,204,0.543103,0.200637,2.17704
4,174,210,0.563218,0.208068,2.25767
5,174,79,0.591954,0.218684,2.1122
6,172,181 174,0.651877,0.20276,2.6583
7,172,174 50,0.78157,0.2431,2.51276
8,172,181 50,0.750853,0.233546,2.02666
9,174,50 98,0.577586,0.213376,2.12534
